In [1]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError
from datetime import date, datetime

pd.options.mode.chained_assignment = None  # default='warn'

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [2]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH = date(2022, 7, 1)

TARGETNAME = 'placement'

In [3]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [4]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,hnp_Mtq6VeuCan6mCb9lZ2drtbfD1FTcEwn0posRI9KMq48,Sb1w6C77m5sMRVQY70Ft0Wm6hP6sfcofnLr2h-chyBm9LNQ,xz-O4rBbMUaS5JRcbMNczdpbrZ7T3ioSO-ceKbC6DqNjMN...,Icey,4661,1656977400000,452,hnp_Mtq6VeuCan6mCb9lZ2drtbfD1FTcEwn0posRI9KMq48,Icey,228,I,53,33,False,False,True,True
1,asx_D8Lm0t2B1o1R3Z2BpSOfaQg_7QdJtFoBvrcxoo-q0-08,QPvzSqwQb-JfMZQtxZ70iWanvPAUD1kd5pa36rULx4kDCo...,bJuSIWIH7bFXcFJryZwWyaoywd4nAAaWQeNRKd0eIS9J6r...,Xellerex,3378,1657177208756,193,asx_D8Lm0t2B1o1R3Z2BpSOfaQg_7QdJtFoBvrcxoo-q0-08,Xellerex,405,I,204,182,False,False,True,False
2,TTdQLI20SMuncZY88NRHHgNCD5CHQgrhPj12GDATzQMxfBeu,qrsgEF7lO1d0QhWURVkdAVEurYh8nc9mdPJv_sUv8eckcx...,1AYfU2oGBkLPXyk_-EhTnN-85BEsUysY1uhjEMZvxaq3Oh...,uuFaded,5194,1657177425000,1,TTdQLI20SMuncZY88NRHHgNCD5CHQgrhPj12GDATzQMxfBeu,uuFaded,344,I,116,81,False,False,True,False
3,YsfIZVAbroGmQamvWM84NEhlz0qB6ahCz_OIHrXUfhPQdHI,XtPFF5eWcMNIGlXhRGEBWQOa7ITMO_xPt4FId9SLTfXnNQ,VTkKEB8vMbUdZIhVt8qlxsRdoTbG99kJ-EALAwzhCnBlB2...,SagecR,23,1657138589000,114,YsfIZVAbroGmQamvWM84NEhlz0qB6ahCz_OIHrXUfhPQdHI,SagecR,208,I,124,112,False,False,True,True
4,L8ZnO_dEDQYyVL3zf69oHA06mg20D9pfgz1jirkoQ0Fr3NT8,ow6PxI-kQfLEhVECJaHTpK3QXrzBJfuvx_bE5awGESzOBz...,ayhyBBmn8zZTMxBUHvb7IzY7B9sHpcTBYxmMX0TdYH_2jA...,yourrich,5169,1657166313000,230,L8ZnO_dEDQYyVL3zf69oHA06mg20D9pfgz1jirkoQ0Fr3NT8,yourrich,200,I,116,81,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,0EeWt-iA7o1vI6KFyo2c87Q217sLyUe-hP2sAtki_vV5X4w,k6EQ2Gp762ymBLMdGqoobYdJ6ERoi-GLXHOwRd_gmc8WIw,urayq63QvvrNY0FlZd--3hXu4kIh0m0suI3uto6JfUm6wS...,Yiggles,4277,1657167455000,125,0EeWt-iA7o1vI6KFyo2c87Q217sLyUe-hP2sAtki_vV5X4w,Yiggles,416,I,140,106,False,False,True,False
483,09EFThM5TgTXzif_RPaFVfsokwq2WL37K981PxyftyY8cK...,tHzAHp2REzHELqMpYTLV0xUR5QFlZRxHKhH1deem5fQEzp...,2h7R8kw9J0blIumXPKQX_eM_oQnVKpjPyRjjEFMYMhSsbw...,SewBerLow,29,1657027042000,1,09EFThM5TgTXzif_RPaFVfsokwq2WL37K981PxyftyY8cK...,SewBerLow,301,I,77,46,False,False,True,True
484,xdi366oQg7491x4QH2Mdqn_lpHIPhETKzmOJMrbuY1uaYvE,gzJZZ75BQD_7hCJbbnZCxFsi0-QrUrusiiz0KMgSP0O38A,kxWE95ML5C_TqrNyTlWp6zSl8DB6x3nOzzccczDdKDy265...,XTheFarmerX,28,1657154751216,195,xdi366oQg7491x4QH2Mdqn_lpHIPhETKzmOJMrbuY1uaYvE,XTheFarmerX,468,I,122,99,False,False,True,True
485,7JdDtGe92I5Gia2DeqExCZI9RbKJbhQ507oHOt3_tIyW1d65,OFytNbs-ZkQhryYFwJYexOxWEg2NW4i9jiXuLw9rbkJogG...,AB2wLDcC047q00ZidhkOvT2bmrzDgPy31MqZUWtt5SWozF...,navioot,3378,1657177374000,1,7JdDtGe92I5Gia2DeqExCZI9RbKJbhQ507oHOt3_tIyW1d65,navioot,255,I,164,140,False,False,True,False


In [5]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

2022-07-07 17:50:18,909 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_CN Ehtel .json.gz'
2022-07-07 17:50:19,350 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Biscuitpiggy.json.gz'
2022-07-07 17:50:19,376 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_howdouplaylol.json.gz'
2022-07-07 17:50:19,424 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Whae.json.gz'
2022-07-07 17:50:19,537 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Smalding.json.gz'
2022-07-07 17:50:19,539 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_MÐB Ysy A.json.gz'
2022-07-07 17:50:19,657 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_KYTumbler.json.gz'
2022-07-07 17:50:19,658 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_TaurusTard.json.gz'
2022-07-07 17:50:19,750 [ERROR] [Errno 2] No such file or directory: '

In [6]:
len(uniq_matches_id)

7533

In [7]:
len(matches_asset)

8880

In [8]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [9]:
len(uniq_matches)

7533

In [10]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [11]:
latest_matches = [match for match in uniq_matches if (
    LATEST_RELEASE in match['info']['game_version'])]


In [12]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [13]:
len(latest_matches)

6958

In [14]:
len(latest_patch_matches)

4094

In [15]:
# latest_matches[0]['info']

In [16]:
from typing import List


def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [17]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [18]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4361396724',
 'placement': 6,
 'augment0': 'TFT6_Augment_Electrocharge2',
 'augment1': 'TFT6_Augment_ForceOfNature',
 'augment2': 'TFT6_Augment_ComponentGrabBag',
 'Set7_Assassin': 1,
 'Set7_Astral': 0,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 0,
 'Set7_Guardian': 0,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Ragewing': 1,
 'Set7_Swiftshot': 1,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'Set7_Whispers': 0,
 'TFT7_Thresh': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Thresh_item0': 'WarmogsArmor',
 'TFT7_Qiyana': 2,
 'TFT7_Shen': 2,
 'TFT7_Varus': 1,
 'TFT7_Varus_item0': 'UnstableConcoction',
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'Redemption',
 'TFT7_Talon': 1,
 'TFT7_Talon_item0': 'ThiefsGloves',
 'TFT7_Talon_item1': 'InfinityEdge',
 'TFT7_Talon_item2': 'JeweledGauntlet',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'Quicksilver',
 'TFT7_Xayah_item1': 'GuinsoosRageblade',
 'TFT7_Xayah_item2': 'InfinityEdge',
 'TFT7_Ornn': 1,
 'TFT7_Or

In [19]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [20]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bruiser,Set7_Cavalier,Set7_Guardian,...,TFT7_TahmKench_item2,TFT7_Thresh_item1,TFT7_Yone_item2,TFT7_Thresh_item2,TFT7_Aatrox_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett_item2,TFT7_Heimerdinger_item2
0,NA1_4361396724,6,TFT6_Augment_Electrocharge2,TFT6_Augment_ForceOfNature,TFT6_Augment_ComponentGrabBag,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4361396724,8,TFT6_Augment_CelestialBlessing2,TFT6_Augment_BandOfThieves2,NaN,0.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4361396724,2,TFT6_Augment_CelestialBlessing2,TFT7_Augment_BruiserEmblem2,TFT7_Augment_BruiserTitanicStrength,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4361396724,7,TFT6_Augment_ThreesCompany,TFT6_Augment_RadiantRelics,TFT6_Augment_PortableForge,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4361396724,3,TFT6_Augment_SunfireBoard,TFT6_Augment_Electrocharge3,TFT6_Augment_PortableForge,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,NA1_4363985993,8,TFT6_Augment_MaxLevel10,TFT6_Augment_Diversify2,TFT7_Augment_BruiserTitanicStrength,0.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55660,NA1_4363985993,7,TFT7_Augment_GuildEmblem2,TFT7_Augment_AxiomArc2,TFT6_Augment_TargetDummies,0.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55661,NA1_4363985993,1,TFT6_Augment_ForceOfNature,TFT6_Augment_Ascension,TFT6_Augment_PortableForge,1.0,NaN,2.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55662,NA1_4363985993,6,TFT6_Augment_RadiantRelics,TFT7_Augment_CannoneerEmblem,TFT6_Augment_CyberneticShell2,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bruiser,Set7_Cavalier,Set7_Guardian,...,TFT7_TahmKench_item2,TFT7_Thresh_item1,TFT7_Yone_item2,TFT7_Thresh_item2,TFT7_Aatrox_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett_item2,TFT7_Heimerdinger_item2
0,NA1_4361396724,6,TFT6_Augment_Electrocharge2,TFT6_Augment_ForceOfNature,TFT6_Augment_ComponentGrabBag,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4361396724,8,TFT6_Augment_CelestialBlessing2,TFT6_Augment_BandOfThieves2,NaN,0.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4361396724,2,TFT6_Augment_CelestialBlessing2,TFT7_Augment_BruiserEmblem2,TFT7_Augment_BruiserTitanicStrength,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4361396724,7,TFT6_Augment_ThreesCompany,TFT6_Augment_RadiantRelics,TFT6_Augment_PortableForge,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4361396724,3,TFT6_Augment_SunfireBoard,TFT6_Augment_Electrocharge3,TFT6_Augment_PortableForge,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,NA1_4363985993,8,TFT6_Augment_MaxLevel10,TFT6_Augment_Diversify2,TFT7_Augment_BruiserTitanicStrength,0.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55660,NA1_4363985993,7,TFT7_Augment_GuildEmblem2,TFT7_Augment_AxiomArc2,TFT6_Augment_TargetDummies,0.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55661,NA1_4363985993,1,TFT6_Augment_ForceOfNature,TFT6_Augment_Ascension,TFT6_Augment_PortableForge,1.0,NaN,2.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55662,NA1_4363985993,6,TFT6_Augment_RadiantRelics,TFT7_Augment_CannoneerEmblem,TFT6_Augment_CyberneticShell2,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [22]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [23]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

# Output dataframes

In [24]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)

# Preprocessing

In [25]:
raw_df = matches_league_df.copy()
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("object"):
        df[name] = df[name].fillna("None")
    return df
raw_df = impute(raw_df)

In [43]:
match_id_df = raw_df['match_id']
X = raw_df.drop(['match_id'], axis=1)
y = X.pop(TARGETNAME)
X.fillna('', inplace=True)
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
traits_col = [s for s in numeric_cols if "Set7" in s]
units_col = [s for s in numeric_cols if "TFT7" in s]
augments_col = ['augment0', 'augment1', 'augment2']
items_col = [s for s in categorical_cols if s not in augments_col]
df_unique = X.nunique().to_frame().reset_index()
df_unique.columns = ['Variable','DistinctCount']
unique_items_set = {y for col in items_col for y in X[col].unique().tolist()}
unique_augments_set = {y for col in augments_col for y in X[col].unique().tolist()}
X[f'items_count'] = X[items_col].apply(lambda row: sum(x != 'None' for x in row), axis=1)
X[f'traits_sum'] = X[traits_col].sum(axis=1)
X[f'units_sum'] = X[units_col].sum(axis=1)
X.iloc[X[f'units_sum'].idxmax()]

augment0          TFT7_Augment_MageConference
augment1              TFT6_Augment_ThriftShop
augment2             TFT6_Augment_SecondWind1
Set7_Assassin                             0.0
Set7_Astral                               1.0
                             ...             
TFT7_Zoe_item1                           None
TFT7_Zoe_item2                           None
items_count                                12
traits_sum                                9.0
units_sum                                30.0
Name: 21889, Length: 270, dtype: object

In [27]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
X[numeric_cols] = X[numeric_cols].applymap(np.int64)
matches_df = X.copy()
matches_df[TARGETNAME] = y

In [85]:
def get_unit_items_ranking(df: matches_df, unit: str):
    # filter and melt the dataframe
    df = df.filter(regex=f'placement|{unit}_item0|{unit}_item1|{unit}_item2')
    df[f'unit'] = f'{unit}' # fill in current unit
    # join 3 items to 1 column
    df[f'{unit}_items'] = df[[f'{unit}_item0', f'{unit}_item1', f'{unit}_item2']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
    # sort items for unique combination
    df[f'{unit}_items'] = df[f'{unit}_items'].apply(lambda x: ', '.join(sorted(x.split(', '))))
    df = df.filter(regex=f'placement|{unit}_items|unit')
    m = df.melt(
        ['placement',f'unit'], value_name=f'{unit}_items_grp') #, value_vars=[f'{unit}_items', f'{unit}']
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'{unit}_items_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'unit', f'{unit}_items_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [86]:
get_unit_items_ranking(df = matches_df[:2], unit='TFT7_Zoe')

,unit,TFT7_Zoe_items_grp,value_count,average_placement
0,TFT7_Zoe,"None, None, None",2,7.0


In [87]:
# Get top5 
top5_items_list = []
for unit in units_col:
    df = get_unit_items_ranking(df = matches_df, unit=unit)
    df = df[df['value_count']>=12][:5] #Top 5 with counts >= 12
    top5_items_list.extend(df.values)

In [88]:
top5_items_list = pd.DataFrame(top5_items_list, columns=['unit', 'items',	'value_count',	'average_placement'])

In [31]:
top5_items_list.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_top5_items.csv'), index=False)

In [32]:
pd.options.display.max_colwidth = 100

In [89]:
top5_items_list.groupby('unit').head(1)

,unit,items,value_count,average_placement
0,TFT7_Aatrox,"None, None, Zephyr",13,2.230769
5,TFT7_Anivia,"ArchangelsStaff, GuinsoosRageblade, Morellonomicon",18,2.666667
10,TFT7_AoShin,"ArchangelsStaff, SpearOfShojin, SpearOfShojin",12,3.000000
15,TFT7_Ashe,"StatikkShiv, ZekesHerald, ZekesHerald",14,2.857143
20,TFT7_AurelionSol,"ArchangelsStaff, HextechGunblade, MageEmblemItem",31,4.064516
24,TFT7_Bard,"None, None, ThiefsGloves",13,1.153846
29,TFT7_Braum,"None, None, TitansResolve",14,3.500000
34,TFT7_Corki,"Deathblade, GuinsoosRageblade, StatikkShiv",13,3.153846
39,TFT7_Diana,"FrozenHeart, HextechGunblade, RedBuff",14,2.857143
44,TFT7_DragonBlue,"Quicksilver, RevelEmblemItem, StatikkShiv",21,2.714286


In [184]:
def get_unit_comp_ranking(df: matches_df):
    # filter and melt the dataframe
    df = df.filter(['placement']+units_col)
    # join units to 1 column
    df['comp'] = df[units_col].apply(lambda row: ', '.join(row[row > 1].index.values.astype(str)), axis=1)
    df['comp'] = df['comp'].str.replace('TFT7_','') #.split('_',1).str[-1]
    df = df.filter(['placement', 'comp'])
    m = df.melt(
        ['placement'], value_name=f'comp_grp')
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'comp_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'comp_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [141]:
# get_unit_comp_ranking(df = matches_df[:50])

In [185]:
# Get top5 
df = get_unit_comp_ranking(df = matches_df)

In [186]:
top5_comp_list = []
m = df[df['value_count']>=80] #[:5] #Top 5 with counts >= 12
top5_comp_list.extend(m.values)
top5_comp_list = pd.DataFrame(top5_comp_list, columns=['comp', 'value_count', 'average_placement'])

In [187]:
top5_comp_list.groupby('comp').head(1)

,comp,value_count,average_placement
0,"Bard, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah",104,1.653846
1,"Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo",105,1.666667
2,"Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah",838,2.830549
3,"Diana, Olaf, Ornn, Qiyana, Shen, Sylas, Talon",136,3.022059
4,"DragonPurple, Ornn, Qiyana, Shen, Sylas, Talon",96,3.145833
5,"Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko",94,3.148936
6,"Hecarim, Ornn, Qiyana, Shen, Talon, Twitch, Xayah",343,3.218659
7,"Anivia, DragonGreen, Gnar, Lulu, Neeko, Talon",104,3.221154
8,"Ashe, DragonGreen, Ezreal, Karma, LeeSin, Leona, Taric",91,3.230769
9,"Hecarim, Ornn, Sejuani, Shen, Talon, Twitch, Xayah",269,3.360595


In [157]:
top5_comp_list

,comp,value_count,average_placement
0,"Anivia, Bard, Corki, DragonGreen, Gnar, Lulu, Neeko, Sona",2,1.0
1,"Bard, DragonGreen, Gnar, Neeko, Sejuani, Soraka, Talon, Twitch",2,1.0
2,"Bard, DragonGreen, Karma, LeeSin, Neeko, Ornn, Soraka, Yasuo",2,1.0
3,"Braum, Corki, DragonGold, Lulu, Sona, TrainerDragon, Tristana, Yasuo, Zoe",2,1.0
4,"AoShin, DragonGold, DragonPurple, Ornn, Shyvana",2,1.0
